In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils
import torch.utils.data
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import numpy as np

from scipy.interpolate import make_interp_spline, RegularGridInterpolator

from functools import partial

In [ ]:
from pinn import BasePinn

In [ ]:
torch.set_default_device('cuda')

In [ ]:
class StokesPinn(BasePinn):
    def __init__(
        self, num_boundary_samples=64, num_interior_samples=512, device="cuda", seed=0
    ):
        self.num_outputs = 3 # pressure, velx, vely

        super().__init__(self.num_outputs, num_boundary_samples, num_interior_samples, device, seed, regularization_param=100.0)

        self.num_inflow_samples = num_boundary_samples // 4
        self.num_wall_samples = num_boundary_samples // 2
        self.num_outflow_samples = num_boundary_samples - self.num_inflow_samples - self.num_wall_samples

        self.lambda_u = 10e-3 # 10e-10


        #Cached values
        self.cached_boundary = self.sample_boundary_points(use_cached=False)


    def inflow_solution(self, n_samples):
        return torch.ones((n_samples)) * self.p_0

    def outflow_solution(self, n_samples):
        return torch.ones((n_samples)) * self.p_L

    def sample_boundary_points(self, use_cached=False):
        
        if use_cached and self.cached_boundary is None:
            raise RuntimeError("No cached boundary points")

        if use_cached:
            return self.cached_boundary

        inflow = self.sample_inflow_points(self.num_inflow_samples)
        walls = self.sample_wall_points(self.num_wall_samples)
        outflow = self.sample_outflow_points(self.num_outflow_samples)

        return torch.cat([inflow, walls, outflow])


    def compute_boundary_error(self, use_cached=False):
        inflow = self.sample_inflow_points(self.num_inflow_samples)
        walls = self.sample_wall_points(self.num_wall_samples)
        outflow = self.sample_outflow_points(self.num_outflow_samples)
    
        pressure_inflow_pred = self.eval(inflow)[:, 0]
        pressure_outflow_pred = self.eval(outflow)[:, 0]
        vel_walls_pred = self.eval(walls)[:, 1]
    
        inflow_err = pressure_inflow_pred - self.inflow_solution(self.num_inflow_samples)
        outflow_err = pressure_outflow_pred - self.outflow_solution(self.num_outflow_samples)
        vel_err = vel_walls_pred

        # print("pred", pressure_inflow_pred)
        # print("solu", self.inflow_solution(self.num_inflow_samples))
            
        return 10e3 * torch.cat([inflow_err, outflow_err, vel_err])

        

    # Compute error with some input/output pair e.g. on the boundaries or as part of supervised training
    def compute_supervised_error(self, points, values):
        out = self.eval(points)
        # error = out - values
        # print(error)
        return out - values

    # Compute error according to the PDE of the interior
    def compute_unsupervised_error(self, points, rho, power_map=None, flow_field=None):
        points.requires_grad = True
        if flow_field is not None or power_map is not None:
            print("WARNING: flow field or power map are not used")

        rho = torch.tensor(rho(points.cpu().detach().numpy()), dtype=torch.float32).to(self.device).T

        flow = self.eval(points)


        pressure = flow[:, 0]
        grad_pressure = torch.autograd.grad(pressure.sum(), points, create_graph=True)[0]

        u = flow[:, 1:]

        grad_u = torch.autograd.grad(u.sum(), points, create_graph=True)[0]

        grad_u_x = grad_u[:, 0]
        grad_u_y = grad_u[:, 1]

        grad_grad_u_x = torch.autograd.grad(grad_u_x.sum(), points, create_graph=True)[0]
        grad_grad_u_y = torch.autograd.grad(grad_u_y.sum(), points, create_graph=True)[0]

        grad_u_xx = grad_grad_u_x[:, 0]      
        grad_u_yy = grad_grad_u_y[:, 1]   

        div_eps_u = (torch.concat((torch.sum(grad_grad_u_x, dim=1),torch.sum(grad_grad_u_y, dim =1)))).reshape(-1,2)


        alpha_rho = (1 - rho) / (1 + self.q_k * rho) * (self.alpha_s - self.alpha_f) + self.alpha_f
        alpha_rho = alpha_rho.reshape((alpha_rho.shape[0], 1))


        termA = -grad_pressure
        termB = 2.0 / self.Re * div_eps_u 
        termC = - 1.0 / self.Re * (5.0 * self.L**2 / (2.0 * self.H_t**2) + alpha_rho) * u

        # print("A", termA.shape, "B", termB.shape, "C", termC.shape)

        stokes = termA + termB + termC

        u_div = grad_u_xx + grad_u_yy
        
        return torch.cat([stokes[:, 0], stokes[:, 1], u_div], 0)

In [ ]:
num_x, num_y = 100, 100

# u_x = np.loadtxt("./formatted_data99/results-99.h5_vel_0.out").reshape(num_x, num_y)
# u_y = np.loadtxt("./formatted_data99/results-99.h5_vel_1.out").reshape(num_x, num_y)
# u = np.stack([u_x, u_y], -1)

# p_map = np.loadtxt("./formatted_data99/powermap.out").reshape(num_x, num_y)

rho = np.loadtxt("./formatted_data99/results-99.h5_rho.out").reshape(num_x, num_y)



In [ ]:
def interpolate(points_out, value=None, num_x=100, num_y=100):
    x = np.arange(0 + 0.5 / num_x, 1.0, 1.0 / num_x)
    y = np.arange(0 + 0.5 / num_y, 1.0, 1.0 / num_y)

    assert value is not None

    
    interp = RegularGridInterpolator((x, y), value, bounds_error=False, fill_value=None)

    if len(value.shape) == 2:
        return interp(points_out).T

    if len(value.shape) == 3:
        return interp(points_out).T

In [ ]:
num_x, num_y = 100, 100
x = np.arange(0 + 0.5 / num_x, 1.0, 1.0 / num_x)
y = np.arange(0 + 0.5 / num_y, 1.0, 1.0 / num_y)
X, Y = np.meshgrid(x, y)
points = np.array([X.flatten(), Y.flatten()]).T

In [ ]:
# output = interpolate(points, u).T.reshape(num_x, num_y, 2).transpose((1, 0, 2))
# print(output.shape)
# plt.quiver(x, y, u_x, u_y)
# plt.show()
# plt.quiver(x, y, output[:, :, 0], output[:, :, 1])
# plt.show()

plt.pcolormesh(rho)
plt.show()
plt.pcolormesh(interpolate(points, rho).reshape(num_x, num_y).T)
plt.show()

In [ ]:
pinn = StokesPinn(
    num_boundary_samples=64, 
    num_interior_samples=512, 
    device="cuda", 
    seed=0
)

optimizer_LBFGS = optim.LBFGS(pinn.neural_network.parameters(),
                              lr=float(0.5),
                              max_iter=30000,
                              max_eval=30000,
                              history_size=150,
                              line_search_fn="strong_wolfe",
                              tolerance_change=1.0 * np.finfo(float).eps)


optimizer_LBFGS_SHORT = optim.LBFGS(pinn.neural_network.parameters(),
                              lr=float(0.5),
                              max_iter=1000,
                              max_eval=1000,
                              history_size=150,
                              line_search_fn="strong_wolfe",
                              tolerance_change=1.0 * np.finfo(float).eps)

optimizer_ADAM = optim.Adam(pinn.neural_network.parameters(),
                            lr=float(0.005))



In [ ]:
press = np.loadtxt("./formatted_data99/results-99.h5_press_0.out")
u_x = np.loadtxt("./formatted_data99/results-99.h5_vel_0.out")
u_y = np.loadtxt("./formatted_data99/results-99.h5_vel_0.out")

data_points = torch.tensor(points, dtype=torch.float32).to('cuda')
flow_combined = torch.tensor(np.stack([press, u_x, u_y], -1).transpose((1, 0, 2)).reshape(100*100, 3), dtype=torch.float32).to('cuda')

training_data = DataLoader(
                    TensorDataset(data_points, flow_combined),
                    batch_size=100 * 100,
                    shuffle=False,
                )

In [ ]:
hist = pinn.fit(num_epochs = 1,
         optimizer = optimizer_LBFGS_SHORT,
         data=training_data,
         verbose=True)

# hist = [0.0, 1.0]


In [ ]:
plt.figure(dpi=150)
plt.grid(True, which="both", ls=":")
plt.plot(np.arange(1, len(hist) + 1), hist, label="Train Loss")
# plt.xscale("log")
plt.yscale("log")
plt.legend()

In [ ]:
def plot_comparison(reference):
    
    inputs = pinn.sample_domain(10000)
    pred = pinn.eval(inputs)

    pressure = pred[:, 0].reshape(-1,)
    vel_x = pred[:, 1].reshape(-1,)
    vel_y = pred[:, 2].reshape(-1,)

    # print(pred[:, 0])
    # print("Press", pressure)


    fig, axs = plt.subplots(1, 2, figsize=(16, 8), dpi=150)


    im1 = axs[0].tripcolor(inputs[:, 1].detach().cpu(), inputs[:, 0].detach().cpu(), pressure.detach().cpu(), cmap="jet")
    axs[0].set_xlabel("x")
    axs[0].set_ylabel("y")
    plt.colorbar(im1, ax=axs[0], spacing='proportional')
    axs[0].grid(True, which="both", ls=":")
    axs[0].set_title("Pressure")

    im2 = axs[1].pcolormesh(reference[:, :, 0], cmap="jet")
    axs[1].set_xlabel("x")
    axs[1].set_ylabel("y")
    plt.colorbar(im2, ax=axs[1])
    axs[1].grid(True, which="both", ls=":")
    axs[1].set_title("Pressure Reference")


    plt.show()

    fig, axs = plt.subplots(1, 2, figsize=(16, 8), dpi=150)
    im1 = axs[0].tripcolor(inputs[:, 1].detach().cpu(), inputs[:, 0].detach().cpu(), vel_x.detach().cpu(), cmap="jet")
    axs[0].set_xlabel("x")
    axs[0].set_ylabel("y")
    plt.colorbar(im1, ax=axs[0], spacing='proportional')
    axs[0].grid(True, which="both", ls=":")
    axs[0].set_title("Velocity x")

    im1 = axs[1].pcolormesh(reference[:, :, 1], cmap="jet")
    axs[1].set_xlabel("x")
    axs[1].set_ylabel("y")
    plt.colorbar(im1, ax=axs[1])
    axs[1].grid(True, which="both", ls=":")
    axs[1].set_title("Velocity x ref")

    plt.show()

    fig, axs = plt.subplots(1, 2, figsize=(16, 8), dpi=150)
    im1 = axs[0].tripcolor(inputs[:, 1].detach().cpu(), inputs[:, 0].detach().cpu(), vel_y.detach().cpu(), cmap="jet")
    axs[0].set_xlabel("x")
    axs[0].set_ylabel("y")
    plt.colorbar(im1, ax=axs[0], spacing='proportional')
    axs[0].grid(True, which="both", ls=":")
    axs[0].set_title("Velocity Y")

    im1 = axs[1].pcolormesh(reference[:, :, 2], cmap="jet")
    axs[1].set_xlabel("x")
    axs[1].set_ylabel("y")
    plt.colorbar(im1, ax=axs[1])
    axs[1].grid(True, which="both", ls=":")
    axs[1].set_title("Velocity Y reference")


    plt.show()



    # print(reference.shape)

    # print("Data stats:", np.mean(reference[:, :, 0]), 
    #                      np.std(reference[:, :, 0]), 
    #                      np.mean(reference[:, :, 1]), 
    #                      np.std(reference[:, :, 1]),
    #                      np.mean(reference[:, :, 1]), 
    #                      np.std(reference[:, :, 1]))



In [ ]:
plot_comparison(np.stack([press, u_x, u_y], -1).reshape(100, 100, 3)) 

In [ ]:
rho = np.loadtxt("./formatted_data10/results-29.h5_rho.out").reshape(num_x, num_y)

press = np.loadtxt("./formatted_data10/results-29.h5_press_0.out")
u_x = np.loadtxt("./formatted_data10/results-29.h5_vel_0.out")
u_y = np.loadtxt("./formatted_data10/results-29.h5_vel_0.out")

In [ ]:
hist = pinn.fit(num_epochs = 1,
         optimizer = optimizer_LBFGS,
         rho=partial(interpolate, value=rho),
         use_cached_data=False,
         verbose=True)

In [ ]:
plt.figure(dpi=150)
plt.grid(True, which="both", ls=":")
plt.plot(np.arange(1, len(hist) + 1), hist, label="Train Loss")
plt.yscale("log")
plt.legend()

In [ ]:
plt.pcolormesh(rho)
plt.show()

In [ ]:
plot_comparison(np.stack([press, u_x, u_y], -1).reshape(100, 100, 3))